In [35]:
# The imports

import os
from enum import Enum
from dotenv import load_dotenv
from agents import Agent, Runner, trace, WebSearchTool, gen_trace_id
from pydantic import BaseModel, EmailStr, Field
from typing import List, Optional

from pypdf import PdfReader
from IPython.display import Markdown, display

from openai import OpenAI
import json
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
    

OpenAI API Key exists and begins sk-proj-


In [4]:
# Read the resume
reader = PdfReader("data/Saaniya_Desai_-_.pdf")

resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text


In [5]:
print(resume)

Saaniya Desai
github.com/saaniyadesai, saaniya.desai@gmail.com
E X P E R I E N C E 
May 2024 — Sep 2025 MSc Bioinformatics, The Hughes Group University of Glasgow
• Project title: "A Machine Learning Model for Virus Integration Site V alidation"
• Developed a machine learning model to validate viral integration sites in cancer genomes, enhancing 
detection accuracy for genomic research.
• Analyzed large-scale viral integration datasets to uncover host-virus interaction trends with potential 
therapeutic relevance.
May 2023 — Sep 2024 MSc Precision Medicine, The Le Quesne Group University of Glasgow
• Project title: "Artificial Intelligence in the Analysis of High-Resolution Mesothelioma Tissue Microarrays"
• T rained a self-supervised AI model to discover recurrent morphologies related to patient outcome
• Applied advanced image analysis to support translational oncology research.
Jan 2023 — May 2023 BSc Cell Biology Project, The Marston Lab University of Edinburgh 
• Project title: "C

In [60]:
# Create a pydentic model for the resume: include the following fields:
# - name
# - email
# - phone
# - linkedin
# - github
# - list of skills
# - list of experiences
# - list of education
# - list of projects
# - list of certifications
# - list of publications
# - list of patents
# - summary of resume in 2-3 sentences
# - list of all the relevant keywords that can be used to search jobs online

class ExperienceLevel(str, Enum):
    ENTRY = "entry" # 0-2 years
    JUNIOR = "junior" # 2-5 years
    MID = "mid" # 5-10 years    
    SENIOR = "senior" # 10-15 years
    LEAD = "lead" # 15+ years
    EXECUTIVE = "executive" # 20+ years

class Experience(BaseModel):
    title: str = Field(..., description="Job title or position held")
    company: Optional[str] = Field(None, description="Name of the company or organization")
    start_date: Optional[str] = Field(None, description="Start date of the experience")
    end_date: Optional[str] = Field(None, description="End date of the experience")
    description: Optional[str] = Field(None, description="Brief description of responsibilities and achievements")

class Education(BaseModel):
    degree: str = Field(..., description="Degree or qualification obtained")
    institution: Optional[str] = Field(None, description="Name of the educational institution")
    start_date: Optional[str] = Field(None, description="Start date of the education")
    end_date: Optional[str] = Field(None, description="End date of the education")
    description: Optional[str] = Field(None, description="Brief description of coursework or achievements")

class Project(BaseModel):
    name: str = Field(..., description="Name of the project")
    description: Optional[str] = Field(None, description="Brief description of the project")
    link: Optional[str] = Field(None, description="URL or link to the project")

class Certification(BaseModel):
    name: str = Field(..., description="Name of the certification")
    issuer: Optional[str] = Field(None, description="Issuing organization or authority")
    date: Optional[str] = Field(None, description="Date the certification was obtained")

class Publication(BaseModel):
    title: str = Field(..., description="Title of the publication")
    publisher: Optional[str] = Field(None, description="Publisher or journal name")
    date: Optional[str] = Field(None, description="Date of publication")
    link: Optional[str] = Field(None, description="URL or link to the publication")

class Patent(BaseModel):
    title: str = Field(..., description="Title of the patent")
    number: Optional[str] = Field(None, description="Patent number")
    date: Optional[str] = Field(None, description="Date the patent was granted")
    description: Optional[str] = Field(None, description="Brief description of the patent")

class ResumeModel(BaseModel):
    name: str = Field(..., description="Full name of the candidate")
    email: EmailStr = Field(..., description="Email address of the candidate")
    phone: Optional[str] = Field(None, description="Phone number of the candidate")
    linkedin: Optional[str] = Field(None, description="LinkedIn profile URL")
    github: Optional[str] = Field(None, description="GitHub profile URL")
    skills: List[str] = Field(default_factory=list, description="List of skills")
    experiences: List[Experience] = Field(default_factory=list, description="List of professional experiences")
    education: List[Education] = Field(default_factory=list, description="List of educational qualifications")
    projects: List[Project] = Field(default_factory=list, description="List of projects")
    certifications: List[Certification] = Field(default_factory=list, description="List of certifications")
    publications: List[Publication] = Field(default_factory=list, description="List of publications")
    patents: List[Patent] = Field(default_factory=list, description="List of patents")
    experience_level: ExperienceLevel = Field(..., description="Experience level of the candidate based on number of years of experience")
    summary: Optional[str] = Field(None, description="Summary of the resume in 2-3 sentences")
    keywords: List[str] = Field(default_factory=list, description="List of relevant keywords for job search")
    target_company_profile: Optional[str] = Field(None, description="Profile/sector of the target company used for job search")


# Create a pydentic model for the job search results:
# - job title
# - company
# - location
# - description
# - link
# - apply link  
# - Hiring manager name
# - list of all the relevant keywords that matched the job posting

class JobPosting(BaseModel):
    title: str = Field(..., description="Job title or position held")
    company: Optional[str] = Field(None, description="Name of the company or organization")
    location: Optional[str] = Field(None, description="Location of the job posting")
    description: Optional[str] = Field(None, description="Brief description of the job posting")
    link: Optional[str] = Field(None, description="URL or link to the job posting")
    apply_link: Optional[str] = Field(None, description="URL or link to the job application")
    hiring_manager_name: Optional[str] = Field(None, description="Name of the hiring manager")
    keywords: List[str] = Field(default_factory=list, description="List of relevant keywords for the job posting")

class JobPostingList(BaseModel):
    job_postings: List[JobPosting] = Field(default_factory=list, description="List of job postings")

In [7]:
MODEL = "gpt-5-mini"
# MODEL = "gpt-4o-mini"

In [42]:
def system_prompt_4_resume_profiler():
    return f"""
    You are ResumeProfiler, an expert recruiting copilot.

    GOAL
    - Read the included resume of the candidate (plain text extracted from a PDF).
    - Produce resume data into a structured format that conforms to the JSON schema and can be used for job search.
      - Resume data conforms to the following JSON schema represented by the Pydantic model ResumeModel:
        {ResumeModel.model_json_schema()}

    RULES
    - SUMMARY: It is IMPORTANT to produce a concise but accurate hiring-manager summary (5–7 bullet points) of the candidate's resume.
    - KEYWORDS: It is IMPORTANT to produce a list of Boolean search strings that would retrieve relevant roles (AND/OR/quotes/site filters).
    - ExperienceLevel - It is IMPORTANT to produce the experience level of the candidate based on the number of years of experience. 
      - ENTRY if the candidate is just graduating from college (check the education section) or has less than 2 years of experience, 
      - JUNIOR if the candidate has 2-5 years of experience, 
      - MID if the candidate has 5-10 years of experience, 
      - SENIOR if the candidate has 10-15 years of experience, 
      - LEAD if the candidate has 15+ years of experience, 
      -EXECUTIVE if the candidate has 20+ years of experience.
    - Never hallucinate - prefer nulls over guesses.

    OUTPUT
    - Must strictly follow the provided JSON Schema (no extra fields). Please produce only the json scheme, nothing else.
    """

print(system_prompt_4_resume_profiler())


    You are ResumeProfiler, an expert recruiting copilot.

    GOAL
    - Read the included resume of the candidate (plain text extracted from a PDF).
    - Produce resume data into a structured format that conforms to the JSON schema and can be used for job search.
      - Resume data conforms to the following JSON schema represented by the Pydantic model ResumeModel:
        {'$defs': {'Certification': {'properties': {'name': {'description': 'Name of the certification', 'title': 'Name', 'type': 'string'}, 'issuer': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Issuing organization or authority', 'title': 'Issuer'}, 'date': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': 'Date the certification was obtained', 'title': 'Date'}}, 'required': ['name'], 'title': 'Certification', 'type': 'object'}, 'Education': {'properties': {'degree': {'description': 'Degree or qualification obtained', 'title': 'Degree', 'type': 'str

In [43]:
def user_prompt_4_resume_profiler(resume_text):
    return f"""
    Here is the candidate resume text extracted from PDF.
    Analyze ONLY content between the following markers: --- RESUME START --- and --- RESUME END ---.
    --- RESUME START ---
    {resume_text}
    --- RESUME END ---
    """

print(user_prompt_4_resume_profiler(resume))


    Here is the candidate resume text extracted from PDF.
    Analyze ONLY content between the following markers: --- RESUME START --- and --- RESUME END ---.
    --- RESUME START ---
    Saaniya Desai
github.com/saaniyadesai, saaniya.desai@gmail.com
E X P E R I E N C E 
May 2024 — Sep 2025 MSc Bioinformatics, The Hughes Group University of Glasgow
• Project title: "A Machine Learning Model for Virus Integration Site V alidation"
• Developed a machine learning model to validate viral integration sites in cancer genomes, enhancing 
detection accuracy for genomic research.
• Analyzed large-scale viral integration datasets to uncover host-virus interaction trends with potential 
therapeutic relevance.
May 2023 — Sep 2024 MSc Precision Medicine, The Le Quesne Group University of Glasgow
• Project title: "Artificial Intelligence in the Analysis of High-Resolution Mesothelioma Tissue Microarrays"
• T rained a self-supervised AI model to discover recurrent morphologies related to patient out

In [44]:
async def profile_resume(resume_text):
    openai = OpenAI()

    system_prompt = system_prompt_4_resume_profiler()
    user_prompt = user_prompt_4_resume_profiler(resume_text)
    response = openai.chat.completions.parse(
        model=MODEL,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        response_format=ResumeModel
    )
    return response.choices[0].message.parsed


In [45]:
# Use the profile_resume function to profile the resume
profiled_resume_data = await profile_resume(resume)
profiled_resume_data


ResumeModel(name='Saaniya Desai', email='saaniya.desai@gmail.com', phone=None, linkedin=None, github='github.com/saaniyadesai', skills=['Python', 'R', 'SQL', 'Bash', 'Git', 'Docker', 'HPC environments', 'Variant calling', 'Alignment tools', 'Genomic data visualization', 'Single-cell transcriptomics', 'Spatial transcriptomics', 'Multi-omics integration', 'Genomic databases (NCBI, UCSC Table Browser, COSMIC)', 'Machine learning', 'Self-supervised learning', 'Unsupervised learning', 'Snakemake', 'Nextflow', 'Workflow management', 'Containerization', 'Version control'], experiences=[Experience(title='ORISE Fellow', company='United States Food and Drug Administration, Silver Spring, MD', start_date='May 2022', end_date='Aug 2022', description='Conducted biochemical analyses (IC, GC, LC/MS, LC-MS/MS) to support pharmaceutical quality evaluation; authored and presented research on nitrosamine impurity risk mitigation at the 2022 FDA Student Abstracts conference; delivered analytical findings 

In [46]:
print(profiled_resume_data.model_dump_json(indent=4))

{
    "name": "Saaniya Desai",
    "email": "saaniya.desai@gmail.com",
    "phone": null,
    "linkedin": null,
    "github": "github.com/saaniyadesai",
    "skills": [
        "Python",
        "R",
        "SQL",
        "Bash",
        "Git",
        "Docker",
        "HPC environments",
        "Variant calling",
        "Alignment tools",
        "Genomic data visualization",
        "Single-cell transcriptomics",
        "Spatial transcriptomics",
        "Multi-omics integration",
        "Genomic databases (NCBI, UCSC Table Browser, COSMIC)",
        "Machine learning",
        "Self-supervised learning",
        "Unsupervised learning",
        "Snakemake",
        "Nextflow",
        "Workflow management",
        "Containerization",
        "Version control"
    ],
    "experiences": [
        {
            "title": "ORISE Fellow",
            "company": "United States Food and Drug Administration, Silver Spring, MD",
            "start_date": "May 2022",
            "end_da

In [48]:
# Pydentic model for the job search plan
class JobSearchPlan(BaseModel):
    job_search_urls: List[str] = Field(default_factory=list, description="List of job search site URLs to be used for job search. Only return the URLs, no other text.")

JOB_SEARCH_PLANNER_INSTRUCTIONS = f"""
    You are an expert job search assistant. You are given a candidate's resume summary and keywords that represent the job search profile of the candidate and a target company profile.
    Your task is to plan a job search campaign by selecting the most relevant job search sites and creating a list of job search URLs.
    Summary will be enclosed in <SUMMARY>...</SUMMARY> tags.
    Keywords will be enclosed in <KEYWORDS>...</KEYWORDS> tags.
    Target company profile will be enclosed in <TARGET_COMPANY_PROFILE>...</TARGET_COMPANY_PROFILE> tags.
    Return the list of job search URLs in a JSON array format that can be used by the job_search_agent to search for jobs.
    Return only JSON object, no other text.
"""
# Get the job search planner agent
def get_job_search_planner_agent() -> Agent:
    """
    Get the job search planner agent
    @param profiled_resume_data_json: The profiled resume data in JSON format
    @return: Job Search Planner Agent
    """

    job_search_planner_agent = Agent(
        name="JobSearchPlannerAgent",
        instructions=JOB_SEARCH_PLANNER_INSTRUCTIONS,
        model=MODEL,
        output_type=JobSearchPlan,
    )

    return job_search_planner_agent



In [49]:
async def run_job_search_planner_agent(profiled_resume: ResumeModel):
    """
    Run the job search planner agent
    @param profiled_resume_data_json: The profiled resume data in JSON format
    @return: The job search plan
    """

    # Extract summary, keywords and target company profile from the profiled resume data
    resume_summary = profiled_resume.summary
    resume_keywords = profiled_resume.keywords
    company_profile = profiled_resume.target_company_profile

    query = f"""
        Use the following information to plan a job search campaign by selecting the most relevant job search sites and creating a list of job search URLs.
        <SUMMARY>
        {resume_summary}
        </SUMMARY>
        <KEYWORDS>
        {', '.join(resume_keywords)}
        </KEYWORDS>
        <TARGET_COMPANY_PROFILE>
        {company_profile}
        </TARGET_COMPANY_PROFILE>
    """

    print("Creating job search plan...")
    job_search_planner_agent = get_job_search_planner_agent()
    result = await Runner.run(
        job_search_planner_agent, 
        query
    )
    
    print(f"Job search plan created successfully. Found {len(result.final_output.job_search_urls)} job search URLs.")
    return result.final_output_as(JobSearchPlan)


In [50]:
job_search_plan = await run_job_search_planner_agent(profiled_resume_data)
print(job_search_plan)

Creating job search plan...
Job search plan created successfully. Found 20 job search URLs.
job_search_urls=['https://www.linkedin.com/jobs/search/?keywords=bioinformatics%20machine%20learning%20Python%20R%20precision%20medicine%20oncology%20single-cell%20spatial%20transcriptomics', 'https://www.indeed.com/jobs?q=bioinformatics+machine+learning+Python+R+precision+medicine+oncology+single-cell+spatial+transcriptomics', 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=bioinformatics%20machine%20learning%20precision%20medicine%20oncology%20single-cell', 'https://www.biospace.com/jobs/search?keywords=bioinformatics%20machine%20learning%20precision%20medicine%20oncology', 'https://www.nature.com/naturecareers/jobs?query=bioinformatics+machine+learning+precision+medicine+oncology', 'https://jobs.sciencecareers.org/jobs/search?keywords=bioinformatics+machine+learning+precision+medicine+oncology', 'https://wellfound.com/jobs?query=bioinformatics%20machine%20learning%20oncology%20precision%20

In [ ]:
print('\n- '.join(job_search_plan.job_search_urls))

https://www.linkedin.com/jobs/search/?keywords=bioinformatics%20machine%20learning%20Python%20R%20precision%20medicine%20oncology%20single-cell%20spatial%20transcriptomics
- https://www.indeed.com/jobs?q=bioinformatics+machine+learning+Python+R+precision+medicine+oncology+single-cell+spatial+transcriptomics
- https://www.glassdoor.com/Job/jobs.htm?sc.keyword=bioinformatics%20machine%20learning%20precision%20medicine%20oncology%20single-cell
- https://www.biospace.com/jobs/search?keywords=bioinformatics%20machine%20learning%20precision%20medicine%20oncology
- https://www.nature.com/naturecareers/jobs?query=bioinformatics+machine+learning+precision+medicine+oncology
- https://jobs.sciencecareers.org/jobs/search?keywords=bioinformatics+machine+learning+precision+medicine+oncology
- https://wellfound.com/jobs?query=bioinformatics%20machine%20learning%20oncology%20precision%20medicine
- https://www.eurobiojobs.com/?keywords=bioinformatics%20machine%20learning%20oncology
- https://www.massbi

In [61]:
JOB_SEARCHER_INSTRUCTIONS = f"""
    You are an useful assistant, expert in searching jobs online using specified job profile. 
    You are given a job search URLs and a candidate's profile.
    Your task is to search ALL the job search URLs and return a list of job postings.
    RULES:
    - IMPORTANT: You must use the candidate's experience level (enclosed in <EXPERIENCE_LEVEL>...</EXPERIENCE_LEVEL> tags) to filter the job postings.
    - Visit ALL the job search URLs and return a list of ALL possible job postings that match the candidate's profile, keywords and experience level.
    - Return only JSON object, no other text.
"""

def get_job_searcher_agent(search_plan: JobSearchPlan) -> Agent:
    """
    Get the job searcher agent
    @param search_plan: The job search plan
    @return: The job searcher agent
    """
    job_searcher_agent = Agent(
        name="JobSearcherAgent",
        instructions=JOB_SEARCHER_INSTRUCTIONS,
        # tools=[WebSearchTool(filters={"allowed_domains": [search_plan.job_search_urls]})],
        model=MODEL,
        output_type=JobPostingList,
    )
    return job_searcher_agent

In [62]:
async def run_job_searches(user_resume: ResumeModel, search_plan: JobSearchPlan) -> list[JobPosting]:
    """ 
        Perform the job searches to sites in the search plan using candidate's profile - Summary and Keywords.
        @param search_plan: The job search plan
        @return: The list of JobPosting object.
    """
    print("Searching Jobs using the following URLs:")
    for url in search_plan.job_search_urls:
        print(f"Searching {url}")

    job_searcher_agent = get_job_searcher_agent(search_plan)

    print(job_searcher_agent)

    query = f"""
        Search web using candidate's job profile and keywords using specified job search URLs and return a list of job postings.
        User's Job Profile is enclosed in <JOB_PROFILE>...</JOB_PROFILE> tags.
        User's Keywords are enclosed in <KEYWORDS>...</KEYWORDS> tags.
        User's Experience Level is enclosed in <EXPERIENCE_LEVEL>...</EXPERIENCE_LEVEL> tags.
        Job Search URLs are enclosed in <JOB_SEARCH_URLS>...</JOB_SEARCH_URLS> tags.
        <JOB_PROFILE>
        {user_resume.summary}
        </JOB_PROFILE>
        <KEYWORDS>
        {', '.join(user_resume.keywords)}
        </KEYWORDS>
        <EXPERIENCE_LEVEL>
        {user_resume.experience_level}
        </EXPERIENCE_LEVEL>
        <JOB_SEARCH_URLS>
        {', '.join(search_plan.job_search_urls)}
        </JOB_SEARCH_URLS>
        RULES:
        - IMPORTANT: Visit ALL the job search URLs and return a list of ALL possible job postings that match the candidate's profile, keywords and experience level.
        - Return only JSON object, no other text.
    """

    result = await Runner.run(
        job_searcher_agent,
        query
    )
    return result.final_output_as(JobPosting)


In [63]:
job_postings = await run_job_searches(profiled_resume_data, job_search_plan)
print(job_postings)

Searching Jobs using the following URLs:
Searching https://www.linkedin.com/jobs/search/?keywords=bioinformatics%20machine%20learning%20Python%20R%20precision%20medicine%20oncology%20single-cell%20spatial%20transcriptomics
Searching https://www.indeed.com/jobs?q=bioinformatics+machine+learning+Python+R+precision+medicine+oncology+single-cell+spatial+transcriptomics
Searching https://www.glassdoor.com/Job/jobs.htm?sc.keyword=bioinformatics%20machine%20learning%20precision%20medicine%20oncology%20single-cell
Searching https://www.biospace.com/jobs/search?keywords=bioinformatics%20machine%20learning%20precision%20medicine%20oncology
Searching https://www.nature.com/naturecareers/jobs?query=bioinformatics+machine+learning+precision+medicine+oncology
Searching https://jobs.sciencecareers.org/jobs/search?keywords=bioinformatics+machine+learning+precision+medicine+oncology
Searching https://wellfound.com/jobs?query=bioinformatics%20machine%20learning%20oncology%20precision%20medicine
Searchin

In [64]:
print(job_postings.model_dump_json(indent=4))

{
    "job_postings": [
        {
            "title": "Bioinformatics Analyst I (Entry-Level)",
            "company": "National Institutes of Health (NIH)",
            "location": "Bethesda, MD / Hybrid",
            "description": "Entry-level bioinformatics analyst role supporting cancer genomics projects; responsibilities include variant calling pipelines, alignment, single-cell and spatial transcriptomics analysis, and developing Snakemake workflows. Suitable for MSc-level candidates and early-career researchers.",
            "link": "https://www.jobs.nih.gov/search/?k=bioinformatics+machine+learning+oncology",
            "apply_link": "https://www.jobs.nih.gov/search/?k=bioinformatics+machine+learning+oncology",
            "hiring_manager_name": null,
            "keywords": [
                "bioinformatics",
                "variant calling",
                "alignment",
                "single-cell",
                "spatial transcriptomics",
                "snakemake",


In [ ]:
# Orchestrator to run Job Seeker agents with tools
async def run() -> list[JobPosting]:
    """ 
    Run the job search process, yielding the job Listing for the candidate.
    @param query: The query to run the research on.
    """
    trace_id = gen_trace_id()
    with trace("Job Seeker trace", trace_id=trace_id):
        print(f"View trace: https://platform.openai.com/traces/trace?trace_id={trace_id}")
        
        print(f"Profiling resume...")
        profiled_resume = await profile_resume(resume)

        print("Starting Job Search...")
        search_plan = await run_job_search_planner_agent(profiled_resume)

        print("Job Searches planned, starting to search...")     
        search_results = await run_job_searches(profiled_resume, search_plan)

        print("Job Searches complete, writing report...")
        return search_results

In [ ]:
jobs_list = await run()
jobs_list